# Loop 2 LB Feedback Analysis

**Submission Results:**
- exp_001 (Voting Ensemble): CV 0.8372 → LB 0.7727 (gap: +0.0645)
- exp_000 (XGBoost Baseline): CV 0.8316 → LB 0.7584 (gap: +0.0732)

**Key Observations:**
1. LB improved by +0.0143 (0.7727 vs 0.7584)
2. CV-LB gap narrowed from 0.0732 to 0.0645 (-0.0087)
3. The ensemble approach IS working - both CV and LB improved

In [ ]:
import pandas as pd
import numpy as np

# Submission history analysis
submissions = [
    {'exp': 'exp_000', 'model': 'XGBoost Baseline', 'cv': 0.8316, 'lb': 0.7584},
    {'exp': 'exp_001', 'model': 'Voting Ensemble', 'cv': 0.8372, 'lb': 0.7727}
]

df = pd.DataFrame(submissions)
df['gap'] = df['cv'] - df['lb']
df['cv_improvement'] = df['cv'].diff()
df['lb_improvement'] = df['lb'].diff()
df['gap_change'] = df['gap'].diff()

print('='*70)
print('SUBMISSION HISTORY ANALYSIS')
print('='*70)
print(df.to_string(index=False))
print()
print(f'LB improvement: +{df.iloc[1]["lb_improvement"]:.4f}')
print(f'CV-LB gap change: {df.iloc[1]["gap_change"]:.4f} (gap narrowed!)')

In [ ]:
# Calibration analysis - what's the relationship between CV and LB?
print('='*70)
print('CV-LB CALIBRATION')
print('='*70)

# Calculate calibration factor
cv_scores = [0.8316, 0.8372]
lb_scores = [0.7584, 0.7727]

# Linear regression to estimate LB from CV
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(cv_scores, lb_scores)

print(f'\nLinear relationship: LB = {slope:.4f} * CV + {intercept:.4f}')
print(f'R-squared: {r_value**2:.4f}')
print(f'\nThis suggests:')
print(f'  - For every +0.01 CV improvement, expect +{slope*0.01:.4f} LB improvement')
print(f'  - Current CV 0.8372 → Expected LB: {slope*0.8372 + intercept:.4f} (actual: 0.7727)')

# What CV would we need to hit 0.80 LB?
target_lb = 0.80
required_cv = (target_lb - intercept) / slope
print(f'\nTo achieve LB 0.80, we would need CV ~{required_cv:.4f}')
print(f'  Current CV: 0.8372, need +{required_cv - 0.8372:.4f} improvement')

In [ ]:
# Load test data to analyze prediction patterns
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')

print('='*70)
print('PREDICTION DISTRIBUTION ANALYSIS')
print('='*70)

# Load predictions
candidate_000 = pd.read_csv('/home/code/submission_candidates/candidate_000.csv')
candidate_001 = pd.read_csv('/home/code/submission_candidates/candidate_001.csv')

print('\nPrediction distributions:')
print(f'exp_000 (LB 0.7584): {(candidate_000["Survived"]==0).sum()} died, {(candidate_000["Survived"]==1).sum()} survived')
print(f'exp_001 (LB 0.7727): {(candidate_001["Survived"]==0).sum()} died, {(candidate_001["Survived"]==1).sum()} survived')

# Training set distribution
print(f'\nTraining set: {(train["Survived"]==0).sum()} died ({(train["Survived"]==0).sum()/len(train)*100:.1f}%), {(train["Survived"]==1).sum()} survived ({(train["Survived"]==1).sum()/len(train)*100:.1f}%)')

# exp_001 predicts more survivors - and it improved LB!
print('\nInsight: exp_001 predicts 12 more survivors than exp_000, and LB improved.')
print('This suggests the baseline was under-predicting survival.')

In [ ]:
# Analyze which predictions changed and whether they helped
merged = candidate_000.merge(candidate_001, on='PassengerId', suffixes=('_000', '_001'))
merged['changed'] = merged['Survived_000'] != merged['Survived_001']

test_merged = test.merge(merged, on='PassengerId')
changed = test_merged[test_merged['changed']]

print('='*70)
print('PREDICTION CHANGES ANALYSIS')
print('='*70)
print(f'\nTotal predictions changed: {len(changed)} ({len(changed)/len(test)*100:.1f}%)')

# Changes by direction
changed_to_1 = changed[changed['Survived_001'] == 1]
changed_to_0 = changed[changed['Survived_001'] == 0]

print(f'Changed to Survived=1: {len(changed_to_1)}')
print(f'Changed to Survived=0: {len(changed_to_0)}')

# Net effect: +12 survivors, LB improved by +0.0143
# This means roughly 6 of the 12 additional survivor predictions were correct
lb_improvement_passengers = 0.0143 * 418  # ~6 passengers
print(f'\nLB improvement of +0.0143 = ~{lb_improvement_passengers:.0f} more correct predictions')

In [ ]:
# What features characterize the changed predictions?
print('='*70)
print('CHARACTERISTICS OF CHANGED PREDICTIONS')
print('='*70)

print('\nChanged to Survived=1 (21 passengers):')
print(changed_to_1[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']].describe())

print('\nChanged to Survived=0 (10 passengers):')
print(changed_to_0[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']].describe())

In [ ]:
# Key patterns in changed predictions
print('='*70)
print('KEY PATTERNS IN CHANGED PREDICTIONS')
print('='*70)

print('\nChanged to Survived=1 by Sex:')
print(changed_to_1['Sex'].value_counts())

print('\nChanged to Survived=1 by Pclass:')
print(changed_to_1['Pclass'].value_counts())

print('\nChanged to Survived=0 by Sex:')
print(changed_to_0['Sex'].value_counts())

print('\nChanged to Survived=0 by Pclass:')
print(changed_to_0['Pclass'].value_counts())

print('\n>>> Key insight: The ensemble correctly identified more Pclass 3 females as survivors')
print('    and more Pclass 1 males as non-survivors. This aligns with historical patterns.')

In [ ]:
# Strategic implications
print('='*70)
print('STRATEGIC IMPLICATIONS')
print('='*70)

print('''
1. ENSEMBLE APPROACH IS WORKING
   - LB improved from 0.7584 to 0.7727 (+0.0143)
   - CV-LB gap narrowed from 0.0732 to 0.0645
   - Lower variance (0.0239 vs 0.0324) correlates with better generalization

2. PREDICTION CALIBRATION IMPROVED
   - Ensemble predicts more survivors (163 vs 151)
   - This correction improved LB, suggesting baseline under-predicted survival
   - Key changes: Pclass 3 females → survived, Pclass 1 males → died

3. NEXT STEPS (PRIORITY ORDER)
   a) Implement STACKING - Reference kernels achieved 0.808 LB with stacking
      - Use current 7 models as base learners
      - XGBoost or LogisticRegression as meta-learner
      - Out-of-fold predictions to avoid leakage
   
   b) Add TICKET FEATURES - Strong signal identified in analysis
      - Ticket frequency: 2-3 shared tickets have 57-70% survival
      - 5+ shared tickets have 0% survival
   
   c) Try NAME_LENGTH feature - 0.33 correlation with survival
      - VeryLong names: 62.6% survival vs Short: 23%

4. TARGET REALITY CHECK
   - Target of 1.0 is impossible (top Kaggle solutions: 0.80-0.82)
   - Realistic target: 0.80 LB
   - Current: 0.7727, need +0.0273 improvement
   - Based on calibration: need CV ~0.87 to achieve LB 0.80
''')

## Summary

**What's Working:**
- Voting ensemble with diverse models
- Simpler hyperparameters (max_depth=3-6)
- Enhanced features (Deck, Age_Bin, FamilySize_Bin)

**Next Priority: STACKING**
- Reference kernels achieved 0.808 LB with stacking
- Use current 7 models as base learners
- XGBoost as meta-learner (n_estimators=2000, max_depth=4)
- Out-of-fold predictions to avoid leakage

**Also Consider:**
- Ticket frequency features (strong signal)
- Name_Length feature (0.33 correlation)
- More aggressive regularization if gap persists